In [1]:
# !gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo
!gdown 1mq9CWT9MXsH1Y-ihxVL4uEzufQjpBoer

Downloading...
From: https://drive.google.com/uc?id=1mq9CWT9MXsH1Y-ihxVL4uEzufQjpBoer
To: /kaggle/working/processed_data.zip
100%|███████████████████████████████████████| 2.86M/2.86M [00:00<00:00, 214MB/s]


In [2]:
!rm -rf processed_data

!unzip -q processed_data.zip

In [3]:
!pip install -q transformers[torch] accelerate -U

!pip install -q datasets

!pip install -q transformers

!pip install -q iterative-stratification

!pip install -q auto-gptq optimum bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 68.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 112.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 59.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 93.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.3 MB/s eta 0:00:00

In [4]:
import numpy as np

import pandas as pd

import os

from os import path

import glob

from tqdm.notebook import tqdm

from tqdm import tqdm

import string

import copy

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy,BitsAndBytesConfig

from datasets import Dataset,load_dataset, DatasetDict

# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import classification_report,accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support

from sklearn.utils import shuffle

from sklearn.preprocessing import LabelEncoder

from scipy.special import expit

from peft import prepare_model_for_kbit_training

from peft import LoraConfig, get_peft_model,PeftModel


In [5]:
import torch

import torch.nn as nn

import torch.nn.functional as F

In [6]:


models_list_3=[

      "FacebookAI/xlm-roberta-large",

#      "google-bert/bert-base-multilingual-uncased",

          ]



models_name_3=[

"XLM-Roberta-large",

# "bert",

]


In [7]:
kind="train"

task="track_a"

langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]

# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"



# train_data=pd.read_csv(processed_path)


In [8]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    if kind=="dev":
       processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)




not found: processed_data/train/track_a/sum.csv


In [9]:
all_csv

,id,anger,disgust,fear,joy,sadness,lang,clean_message,surprise
0,afr_train_track_a_00001,0,0.0,0,1,0,afr,ons sal gesondheidsprogramme in skole herinstel,NaN
1,afr_train_track_a_00002,0,0.0,0,1,0,afr,ons moet verstaan dat ons in landbou een groot...,NaN
2,afr_train_track_a_00003,0,0.0,0,1,0,afr,hulle is die trots van die boland en hulle hoo...,NaN
3,afr_train_track_a_00004,0,0.0,0,0,0,afr,ek sou nog baie voorbeelde vir u kon noem,NaN
4,afr_train_track_a_00005,0,0.0,0,1,0,afr,in ons bedryf loon dit gewoonlik om geduldig t...,NaN
...,...,...,...,...,...,...,...,...,...
25557,tir_train_track_a_03677,0,0.0,0,0,1,tir,ሕማቅ ስማዕ ጽንዓት ይሃብ ቤተስብን ምቅርብ መቃልስቱን መንግስተ ሰማይ የ...,0.0
25558,tir_train_track_a_03678,1,0.0,0,0,0,tir,ኣጃንዳ ክህቡና ደልዮም ዮም ዞም ሌያቡ,0.0
25559,tir_train_track_a_03679,0,0.0,0,0,0,tir,ኣብ መንጎ ኤርትራን ኢትዮጵያን ተፈጥሩ ዘሎ ሰላም ዝንቅንቆ የለን erit...,0.0
25560,tir_train_track_a_03680,0,0.0,0,0,0,tir,ብሪጣንያ ካብ ፈረንሳይ ዝሰግሩ ስደተኛታት ንምሓዝ ድሮንን ራዳርን ክጥቀም...,0.0


In [10]:
all_csv[all_csv["lang"]=="eng"]

,id,anger,disgust,fear,joy,sadness,lang,clean_message,surprise
7374,eng_train_track_a_00001,0,NaN,0,1,1,eng,but not very happy,0.0
7375,eng_train_track_a_00002,0,NaN,0,1,0,eng,well shes not gon na last the whole song like ...,0.0
7376,eng_train_track_a_00003,0,NaN,0,0,0,eng,she sat at her papas recliner sofa only to mov...,0.0
7377,eng_train_track_a_00004,1,NaN,1,0,1,eng,yes the oklahoma city bombing,1.0
7378,eng_train_track_a_00005,0,NaN,0,1,0,eng,they were dancing to bolero,0.0
...,...,...,...,...,...,...,...,...,...
10137,eng_train_track_a_02764,0,NaN,1,0,0,eng,yeah but did you just find that,1.0
10138,eng_train_track_a_02765,0,NaN,0,0,0,eng,i did as little as possible with my right hand...,0.0
10139,eng_train_track_a_02766,1,NaN,0,0,1,eng,okay that sucks right,0.0
10140,eng_train_track_a_02767,0,NaN,1,0,0,eng,the spark leaped through his body into mine an...,1.0


In [11]:
# float_columns = all_csv.select_dtypes(include=['float64']).columns
# # Check each column and convert only if there are no NaN values
# for col in float_columns:
#     if all_csv[col].isna().sum() == 0:  # No NaN values in the column
#         print(f"converting {kind} {task} {lang} column {col} to int")
#         all_csv[col] = all_csv[col].astype('int')
#     else:
#         print(f"converting {kind} {task} {lang} column {col} to int")

In [12]:
train_data=all_csv


In [13]:
# train_data = train_data.drop(columns=['disgust'],axis=1)


In [14]:

train_data=train_data.fillna(0)

train_data

,id,anger,disgust,fear,joy,sadness,lang,clean_message,surprise
0,afr_train_track_a_00001,0,0.0,0,1,0,afr,ons sal gesondheidsprogramme in skole herinstel,0.0
1,afr_train_track_a_00002,0,0.0,0,1,0,afr,ons moet verstaan dat ons in landbou een groot...,0.0
2,afr_train_track_a_00003,0,0.0,0,1,0,afr,hulle is die trots van die boland en hulle hoo...,0.0
3,afr_train_track_a_00004,0,0.0,0,0,0,afr,ek sou nog baie voorbeelde vir u kon noem,0.0
4,afr_train_track_a_00005,0,0.0,0,1,0,afr,in ons bedryf loon dit gewoonlik om geduldig t...,0.0
...,...,...,...,...,...,...,...,...,...
25557,tir_train_track_a_03677,0,0.0,0,0,1,tir,ሕማቅ ስማዕ ጽንዓት ይሃብ ቤተስብን ምቅርብ መቃልስቱን መንግስተ ሰማይ የ...,0.0
25558,tir_train_track_a_03678,1,0.0,0,0,0,tir,ኣጃንዳ ክህቡና ደልዮም ዮም ዞም ሌያቡ,0.0
25559,tir_train_track_a_03679,0,0.0,0,0,0,tir,ኣብ መንጎ ኤርትራን ኢትዮጵያን ተፈጥሩ ዘሎ ሰላም ዝንቅንቆ የለን erit...,0.0
25560,tir_train_track_a_03680,0,0.0,0,0,0,tir,ብሪጣንያ ካብ ፈረንሳይ ዝሰግሩ ስደተኛታት ንምሓዝ ድሮንን ራዳርን ክጥቀም...,0.0


In [15]:
float_columns = train_data.select_dtypes(include=['float64']).columns

# Convert those columns to int
train_data[float_columns] = train_data[float_columns].astype('int')
train_data

,id,anger,disgust,fear,joy,sadness,lang,clean_message,surprise
0,afr_train_track_a_00001,0,0,0,1,0,afr,ons sal gesondheidsprogramme in skole herinstel,0
1,afr_train_track_a_00002,0,0,0,1,0,afr,ons moet verstaan dat ons in landbou een groot...,0
2,afr_train_track_a_00003,0,0,0,1,0,afr,hulle is die trots van die boland en hulle hoo...,0
3,afr_train_track_a_00004,0,0,0,0,0,afr,ek sou nog baie voorbeelde vir u kon noem,0
4,afr_train_track_a_00005,0,0,0,1,0,afr,in ons bedryf loon dit gewoonlik om geduldig t...,0
...,...,...,...,...,...,...,...,...,...
25557,tir_train_track_a_03677,0,0,0,0,1,tir,ሕማቅ ስማዕ ጽንዓት ይሃብ ቤተስብን ምቅርብ መቃልስቱን መንግስተ ሰማይ የ...,0
25558,tir_train_track_a_03678,1,0,0,0,0,tir,ኣጃንዳ ክህቡና ደልዮም ዮም ዞም ሌያቡ,0
25559,tir_train_track_a_03679,0,0,0,0,0,tir,ኣብ መንጎ ኤርትራን ኢትዮጵያን ተፈጥሩ ዘሎ ሰላም ዝንቅንቆ የለን erit...,0
25560,tir_train_track_a_03680,0,0,0,0,0,tir,ብሪጣንያ ካብ ፈረንሳይ ዝሰግሩ ስደተኛታት ንምሓዝ ድሮንን ራዳርን ክጥቀም...,0


In [16]:
train_data.dtypes

id               object
anger             int64
disgust           int64
fear              int64
joy               int64
sadness           int64
lang             object
clean_message    object
surprise          int64
dtype: object

In [17]:
train_data = train_data[

train_data['clean_message'].isnull()==False]

train_data.reset_index()

,index,id,anger,disgust,fear,joy,sadness,lang,clean_message,surprise
0,0,afr_train_track_a_00001,0,0,0,1,0,afr,ons sal gesondheidsprogramme in skole herinstel,0
1,1,afr_train_track_a_00002,0,0,0,1,0,afr,ons moet verstaan dat ons in landbou een groot...,0
2,2,afr_train_track_a_00003,0,0,0,1,0,afr,hulle is die trots van die boland en hulle hoo...,0
3,3,afr_train_track_a_00004,0,0,0,0,0,afr,ek sou nog baie voorbeelde vir u kon noem,0
4,4,afr_train_track_a_00005,0,0,0,1,0,afr,in ons bedryf loon dit gewoonlik om geduldig t...,0
...,...,...,...,...,...,...,...,...,...,...
25557,25557,tir_train_track_a_03677,0,0,0,0,1,tir,ሕማቅ ስማዕ ጽንዓት ይሃብ ቤተስብን ምቅርብ መቃልስቱን መንግስተ ሰማይ የ...,0
25558,25558,tir_train_track_a_03678,1,0,0,0,0,tir,ኣጃንዳ ክህቡና ደልዮም ዮም ዞም ሌያቡ,0
25559,25559,tir_train_track_a_03679,0,0,0,0,0,tir,ኣብ መንጎ ኤርትራን ኢትዮጵያን ተፈጥሩ ዘሎ ሰላም ዝንቅንቆ የለን erit...,0
25560,25560,tir_train_track_a_03680,0,0,0,0,0,tir,ብሪጣንያ ካብ ፈረንሳይ ዝሰግሩ ስደተኛታት ንምሓዝ ድሮንን ራዳርን ክጥቀም...,0


In [18]:
train_data

,id,anger,disgust,fear,joy,sadness,lang,clean_message,surprise
0,afr_train_track_a_00001,0,0,0,1,0,afr,ons sal gesondheidsprogramme in skole herinstel,0
1,afr_train_track_a_00002,0,0,0,1,0,afr,ons moet verstaan dat ons in landbou een groot...,0
2,afr_train_track_a_00003,0,0,0,1,0,afr,hulle is die trots van die boland en hulle hoo...,0
3,afr_train_track_a_00004,0,0,0,0,0,afr,ek sou nog baie voorbeelde vir u kon noem,0
4,afr_train_track_a_00005,0,0,0,1,0,afr,in ons bedryf loon dit gewoonlik om geduldig t...,0
...,...,...,...,...,...,...,...,...,...
25557,tir_train_track_a_03677,0,0,0,0,1,tir,ሕማቅ ስማዕ ጽንዓት ይሃብ ቤተስብን ምቅርብ መቃልስቱን መንግስተ ሰማይ የ...,0
25558,tir_train_track_a_03678,1,0,0,0,0,tir,ኣጃንዳ ክህቡና ደልዮም ዮም ዞም ሌያቡ,0
25559,tir_train_track_a_03679,0,0,0,0,0,tir,ኣብ መንጎ ኤርትራን ኢትዮጵያን ተፈጥሩ ዘሎ ሰላም ዝንቅንቆ የለን erit...,0
25560,tir_train_track_a_03680,0,0,0,0,0,tir,ብሪጣንያ ካብ ፈረንሳይ ዝሰግሩ ስደተኛታት ንምሓዝ ድሮንን ራዳርን ክጥቀም...,0


In [19]:
label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise']

In [20]:
train_data['lang'].value_counts()

lang
tir     3681
amh     3549
oro     3442
som     3392
eng     2768
rus     2679
deu     2603
ptbr    2226
afr     1222
Name: count, dtype: int64

In [21]:
sum(train_data['lang'].isna())

0

In [22]:
nan_rows = train_data[train_data.isna().any(axis=1)]
nan_rows
# Group by 'lang' and list the rows with NaNs for each language
# nan_by_language = nan_rows.groupby('lang').apply(lambda x: x)

# Display the result
# print(nan_by_language)

,id,anger,disgust,fear,joy,sadness,lang,clean_message,surprise


In [23]:
nan_rows[nan_rows["lang"]=="eng"]

,id,anger,disgust,fear,joy,sadness,lang,clean_message,surprise


In [24]:
train_data[train_data["lang"]=="deu"].dtypes

id               object
anger             int64
disgust           int64
fear              int64
joy               int64
sadness           int64
lang             object
clean_message    object
surprise          int64
dtype: object

In [25]:
for lang in langs:
    for label in label_columns:
        print(lang,label,train_data[train_data["lang"]=="afr"][label].unique())

afr anger [0 1]
afr fear [0 1]
afr joy [1 0]
afr sadness [0 1]
afr surprise [0]
amh anger [0 1]
amh fear [0 1]
amh joy [1 0]
amh sadness [0 1]
amh surprise [0]
deu anger [0 1]
deu fear [0 1]
deu joy [1 0]
deu sadness [0 1]
deu surprise [0]
eng anger [0 1]
eng fear [0 1]
eng joy [1 0]
eng sadness [0 1]
eng surprise [0]
oro anger [0 1]
oro fear [0 1]
oro joy [1 0]
oro sadness [0 1]
oro surprise [0]
ptbr anger [0 1]
ptbr fear [0 1]
ptbr joy [1 0]
ptbr sadness [0 1]
ptbr surprise [0]
rus anger [0 1]
rus fear [0 1]
rus joy [1 0]
rus sadness [0 1]
rus surprise [0]
som anger [0 1]
som fear [0 1]
som joy [1 0]
som sadness [0 1]
som surprise [0]
sum anger [0 1]
sum fear [0 1]
sum joy [1 0]
sum sadness [0 1]
sum surprise [0]
tir anger [0 1]
tir fear [0 1]
tir joy [1 0]
tir sadness [0 1]
tir surprise [0]


In [26]:
nan_rows["lang"].value_counts()

Series([], Name: count, dtype: int64)

In [27]:
nan_rows.dtypes

id               object
anger             int64
disgust           int64
fear              int64
joy               int64
sadness           int64
lang             object
clean_message    object
surprise          int64
dtype: object

In [28]:
# Step 2: Count how many columns are not 0 or 1 for each language
def count_nonzero_nonone_rows(group):
    # Exclude the 'lang' column from counting if needed
    # Use boolean conditions to count values that are not 0 or 1
    return (group.drop(columns='anger') != 0) & (group.drop(columns='anger') != 1).sum(axis=1).sum()

# Group by 'lang' and apply the function to count non-zero, non-one values
non_zero_non_one_counts = train_data.groupby('lang').apply(count_nonzero_nonone_rows)

# Display the results
# result = pd.DataFrame({
#     'row_count': lang_row_counts,
#     'non_zero_non_one_count': non_zero_non_one_counts
# })

print(non_zero_non_one_counts)

              id  disgust   fear    joy  sadness  lang  clean_message  \
lang                                                                    
afr  0      True    False  False   True    False  True           True   
     1      True    False  False   True    False  True           True   
     2      True    False  False   True    False  True           True   
     3      True    False  False  False    False  True           True   
     4      True    False  False   True    False  True           True   
...          ...      ...    ...    ...      ...   ...            ...   
tir  25557  True    False  False  False     True  True           True   
     25558  True    False  False  False    False  True           True   
     25559  True    False  False  False    False  True           True   
     25560  True    False  False  False    False  True           True   
     25561  True    False  False  False    False  True           True   

            surprise  
lang                  
afr 

In [29]:
label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise']


In [30]:
# shuffle data
train_data=train_data.sample(frac=1)
# train_data=train_data.sample(100,random_state=42)


In [31]:
# train valid split
train,valid=train_test_split(train_data,test_size=0.2,random_state=42)

In [32]:
x_train, y_train = train['clean_message'].values.tolist(), train[label_columns].values.tolist()

x_valid, y_valid = valid['clean_message'].values.tolist(), valid[label_columns].values.tolist()

In [33]:
y_train_df = pd.DataFrame(y_train, columns=label_columns)

In [34]:
#update label column name

label_distribution = y_train_df.apply(pd.Series.value_counts).T.fillna(0).astype(int)

label_distribution.columns = ['count_0', 'count_1']
label_distribution['sum'] = label_distribution['count_0'] + label_distribution['count_1']
print(label_distribution)

          count_0  count_1    sum
anger       16363     4086  20449
fear        17946     2503  20449
joy         16031     4418  20449
sadness     16945     3504  20449
surprise    18573     1876  20449


In [35]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [36]:
from sklearn.utils.class_weight import compute_class_weight

counts_0 = label_distribution['count_0'].to_numpy()
counts_1 = label_distribution['count_1'].to_numpy()

# Compute class-specific weights for each label (each class)
class_weights = []
for i in range(len(counts_0)):
    weight = compute_class_weight('balanced', classes=np.array([0, 1]), y=[0] * counts_0[i] + [1] * counts_1[i])
    class_weights.append(weight)

# Convert the list of weights to a tensor
class_weights_tensor = torch.tensor([w[1] for w in class_weights], dtype=torch.float,device=device)
class_weights_tensor

tensor([2.5023, 4.0849, 2.3143, 2.9180, 5.4502], device='cuda:0')

In [37]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, encodings, labels):

        self.encodings = encodings

        self.labels = labels



    def __getitem__(self, idx):

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)

        return item


    def __len__(self):

        return len(self.labels)

In [38]:
!ls output/best-model-v1

ls: cannot access 'output/best-model-v1': No such file or directory


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [39]:
def build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH):

  train_encodings = tokenizer(x_train, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  valid_encodings = tokenizer(x_valid, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  train_dataset = Dataset(train_encodings, y_train)

  valid_dataset = Dataset(valid_encodings, y_valid)

  return train_dataset,valid_dataset



def train_arg():

    training_args = TrainingArguments(

    output_dir="./output/best-model-v1",

    report_to=None,
    dataloader_pin_memory=False,

    num_train_epochs=3,

    gradient_accumulation_steps=2,

    per_device_train_batch_size=8,

    per_device_eval_batch_size=16,

    learning_rate=2e-5,

    weight_decay=0.01,

    eval_strategy=IntervalStrategy.STEPS,

    save_total_limit=1,

    logging_dir="./logs",

    logging_strategy=IntervalStrategy.STEPS,

    logging_steps=250,

    eval_steps=500,

    load_best_model_at_end=True,

    lr_scheduler_type="cosine_with_restarts",

    warmup_steps=500,

    save_strategy="steps",

    save_steps=500,

  )

    return training_args

In [40]:
def compute_metrics(p):

  predicted_logits = p.predictions

  print(f"Type of predicted_logits: {type(predicted_logits)}")

  if isinstance(predicted_logits, tuple):

        predicted_logits = predicted_logits[0]

  if not isinstance(predicted_logits, np.ndarray):

        predicted_logits = np.array(predicted_logits)

  if len(predicted_logits.shape) == 3:

        predicted_logits = predicted_logits[:, 0, :]

  predicted_probabilities = expit(predicted_logits)

  threshold = 0.5

  predicted_classes = (predicted_probabilities >= threshold).astype(int)

  precision, recall, f1, _ = precision_recall_fscore_support(y_valid, predicted_classes, average="weighted")

  accuracy = accuracy_score(y_valid, predicted_classes)

  return {

        "accuracy": accuracy,

        "precision": precision,

        "recall": recall,

        "f1_score": f1

    }

In [41]:
def evaluate_func(trainer):

  trainer.evaluate()

  predictions = trainer.predict(valid_dataset)

  predicted_logits = predictions.predictions

  print(f"Type of predicted_logits: {type(predicted_logits)}")

  if isinstance(predicted_logits, tuple):

        predicted_logits = predicted_logits[0]

  if not isinstance(predicted_logits, np.ndarray):

        predicted_logits = np.array(predicted_logits)

  if len(predicted_logits.shape) == 3:

        predicted_logits = predicted_logits[:, 0, :]

  predicted_probabilities = expit(predicted_logits)

  threshold = 0.5

  predicted_classes = (predicted_probabilities >= threshold).astype(int)

  precision, recall, f1, _ = precision_recall_fscore_support(y_valid, predicted_classes, average=None,zero_division=0)

  accuracy = accuracy_score(y_valid, predicted_classes)

  return precision, recall, f1,accuracy

In [42]:
MAX_LENGTH = 256

metrics_df = pd.DataFrame(columns=["Models_name", "Precision", "Recall", "F1_score", "Accuracy"])

confusion_matrices = []

In [43]:
quantization_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_use_double_quant=True,

    bnb_4bit_quant_type="nf4",

    bnb_4bit_compute_dtype=torch.bfloat16,

)

In [44]:
models_list=models_list_3

models_name=models_name_3

x=models_list[0]

In [45]:
token=""

import huggingface_hub

if "gemma" in x or "Phi" in x:

  huggingface_hub.login(token=token)

In [46]:
USE_LORA=False

In [47]:


torch.cuda.empty_cache()



print(x)

tokenizer = AutoTokenizer.from_pretrained(x)

model = AutoModelForSequenceClassification.from_pretrained(x, num_labels=len(label_columns),

                                                            ignore_mismatched_sizes=True,

                                                            quantization_config=quantization_config if USE_LORA else None,

                                                            # low_cpu_mem_usage=True,

                                                           torch_dtype=torch.bfloat16,

                                                           problem_type = "multi_label_classification",



                                                            )

FacebookAI/xlm-roberta-large


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()

    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear4bit'
        if "Linear4bit" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]

            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)
find_target_modules(model)

[]

In [49]:
# a=

In [50]:
model.train() # model in training mode (dropout modules are activated)

if USE_LORA:

  # enable gradient check pointing

  model.gradient_checkpointing_enable()

  target_modules_dict={

      "microsoft/Phi-3.5-mini-instruct":'all-linear',
      "unsloth/Llama-3.2-1B-Instruct":["q_proj","k_proj","v_proj"],
      "unsloth/Qwen2.5-3B-Instruct-bnb-4bit":["q_proj","k_proj","v_proj"],
  }

  target_modules= target_modules_dict.get(x,["q_proj"])



  # enable quantized training

  model = prepare_model_for_kbit_training(model)

  config = LoraConfig(

    r=8,

    lora_alpha=32,

    # target_modules=["k_proj"],

    target_modules = target_modules,#TODOOOOOOOOOOOOO

    lora_dropout=0.1,

    bias="none",

    task_type="SEQ_CLS",

  )

  # LoRA trainable version of model

  model = get_peft_model(model, config)



  # trainable parameter count

  model.print_trainable_parameters()

device="cuda" if torch.cuda.is_available() else 'cpu'

In [51]:
if tokenizer.pad_token is None:
    print("adding tokenizer token")
    # tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    tokenizer.pad_token=tokenizer.eos_token

    model.config.pad_token_id = tokenizer.pad_token_id

    # model.resize_token_embeddings(len(tokenizer))

    if model.config.pad_token_id is None:
        print("setting model pad token")
        model.config.pad_token_id = tokenizer.pad_token_id

tokenizer.pad_token

'<pad>'

In [52]:
device

'cuda'

In [53]:
model = model.to(device)

# for param in model.parameters():

#     param.data = param.data.contiguous()


In [54]:
os.environ['WANDB_DISABLED']="true"
training_args = train_arg()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [55]:
train_dataset,valid_dataset=build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH)

In [56]:
# Define a custom loss function to handle class weights for multilabel classification
def custom_loss(output, target):
    # BCEWithLogitsLoss with pos_weight for each class
    loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=class_weights_tensor)
    return loss_fn(output, target)

In [57]:
# Define the Trainer with custom loss function
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False,num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = custom_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [58]:
trainer = CustomTrainer(

    model=model,

    args=training_args,

    train_dataset=train_dataset,

    eval_dataset=valid_dataset,

    compute_metrics=compute_metrics,

    # data_collator=data_collator

)

In [59]:
checkpoints = sorted(glob.glob(os.path.join(trainer.args.output_dir, "checkpoint-*")), key=os.path.getmtime)



resume_checkpoint = True if checkpoints else False

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

trainer.train(resume_from_checkpoint=resume_checkpoint)

model.config.use_cache = True


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Score
500,1.735700,0.821779,0.319186,0.279799,0.250559,0.255134
1000,1.584800,0.727486,0.393702,0.478874,0.415694,0.438655
1500,1.398000,0.662836,0.412282,0.488550,0.584554,0.525098
2000,1.299900,0.626390,0.429493,0.504560,0.640179,0.562561
2500,1.296400,0.610106,0.425973,0.499083,0.655327,0.565493
3000,1.247900,0.606903,0.438099,0.508222,0.650608,0.569495
3500,1.267000,0.606862,0.441033,0.511784,0.648870,0.571105


Type of predicted_logits: <class 'numpy.ndarray'>


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>


In [60]:
model.config.use_cache = True
precision, recall, f1,accuracy=evaluate_func(trainer)
new_data = pd.DataFrame({

  "Models_name": [x],

  "Precision": [precision],

  "Recall": [recall],

  "F1_score": [f1],

  "Accuracy": [accuracy]

})



metrics_df = pd.concat([metrics_df, new_data], ignore_index=True)



model_confusion_matrices = []

predicted_classes = trainer.predict(valid_dataset).predictions







if isinstance(predicted_classes, tuple) :

  predicted_classes = predicted_classes[0]

predicted_classes = np.array(predicted_classes)

# print(f"Type after conversion: {type(predicted_classes)}")



if len(predicted_classes.shape) > 2:

  predicted_classes = predicted_classes.reshape(predicted_classes.shape[0], -1)

  # print(f"Shape after reshaping: {predicted_classes.shape}")









predicted_classes = (expit(predicted_classes) >= 0.5).astype(int)



y_valid_np = np.array(y_valid)



for j in range(len(label_columns)):

    print(f"Computing confusion matrix for label: {label_columns[j]}")

    cm = confusion_matrix(y_valid_np[:, j], predicted_classes[:, j])



    # cm = confusion_matrix(y_valid[:, j], predicted_classes[:, j])

    model_confusion_matrices.append(cm)

confusion_matrices.append(model_confusion_matrices)





print(f"{x} evaluates:")

for k, label in enumerate(label_columns):

  print(f"Class '{label}': Precision: {precision[k]:.4f}, Recall: {recall[k]:.4f}, F1: {f1[k]:.4f}")

print(f"Overall accuracy: {accuracy:.4f}")

print("-------------------------------------------------------------------------\n")

Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>


<ipython-input-60-280a069bd3c3>:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_df = pd.concat([metrics_df, new_data], ignore_index=True)


Type of predicted_logits: <class 'numpy.ndarray'>
Computing confusion matrix for label: anger
Computing confusion matrix for label: fear
Computing confusion matrix for label: joy
Computing confusion matrix for label: sadness
Computing confusion matrix for label: surprise
FacebookAI/xlm-roberta-large evaluates:
Class 'anger': Precision: 0.5514, Recall: 0.6045, F1: 0.5767
Class 'fear': Precision: 0.5018, Recall: 0.7069, F1: 0.5870
Class 'joy': Precision: 0.5559, Recall: 0.7267, F1: 0.6299
Class 'sadness': Precision: 0.4378, Recall: 0.5862, F1: 0.5012
Class 'surprise': Precision: 0.4652, Recall: 0.6014, F1: 0.5246
Overall accuracy: 0.4410
-------------------------------------------------------------------------



In [62]:
# Initialize a DataFrame to store the evaluation results for each language
lang_metrics_df = pd.DataFrame(columns=["Language", "Precision", "Recall", "F1_score", "Accuracy"])
# Temporarily disable metric computation
original_compute_metrics = trainer.compute_metrics
trainer.compute_metrics = None

# Iterate over each language
for lang in langs:
    # Filter the validation data for the current language
    lang_valid_data = valid[valid['lang'] == lang]
    lang_x_valid = lang_valid_data['clean_message'].values.tolist()
    lang_y_valid = lang_valid_data[label_columns].values.tolist()
    if(len(lang_valid_data)==0):
      continue
    # Tokenize the validation data
    lang_valid_encodings = tokenizer(lang_x_valid, truncation=True, padding="max_length", max_length=MAX_LENGTH)
    lang_valid_dataset = Dataset(lang_valid_encodings, lang_y_valid)

    # Get predictions
    predictions = trainer.predict(lang_valid_dataset,)
    predicted_logits = predictions.predictions

    if isinstance(predicted_logits, tuple):
        predicted_logits = predicted_logits[0]

    if not isinstance(predicted_logits, np.ndarray):
        predicted_logits = np.array(predicted_logits)

    if len(predicted_logits.shape) == 3:
        predicted_logits = predicted_logits[:, 0, :]

    predicted_probabilities = expit(predicted_logits)
    threshold = 0.5
    predicted_classes = (predicted_probabilities >= threshold).astype(int)

    # Compute metrics
    precision, recall, f1, _ = precision_recall_fscore_support(lang_y_valid, predicted_classes, average="weighted")
    accuracy = accuracy_score(lang_y_valid, predicted_classes)

    # Store the metrics in the DataFrame
    new_data = pd.DataFrame({
        "Language": [lang],
        "Precision": [precision],
        "Recall": [recall],
        "F1_score": [f1],
        "Accuracy": [accuracy]
    })

    lang_metrics_df = pd.concat([lang_metrics_df, new_data], ignore_index=True)

# Display the evaluation results for each language
print(lang_metrics_df)

# Restore compute_metrics function if needed
trainer.compute_metrics = original_compute_metrics

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-62-bf824cdfa0a4>:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  lang_metrics_df = pd.concat([lang_metrics_df, new_data], ignore_index=True)


  Language  Precision    Recall  F1_score  Accuracy
0      afr   0.501880  0.668478  0.560070  0.465909
1      amh   0.534074  0.696809  0.602567  0.478378
2      deu   0.538971  0.670534  0.589932  0.431373
3      eng   0.584194  0.827543  0.648287  0.222868
4      oro   0.474656  0.512088  0.469102  0.476551
5     ptbr   0.550212  0.670241  0.601662  0.467269
6      rus   0.529566  0.753425  0.609762  0.409524
7      som   0.379769  0.516854  0.420146  0.462555
8      tir   0.408208  0.342857  0.364975  0.505821


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
a=

In [63]:
import os
os.environ["HUGGING_FACE_HUB_TOKEN"] = token

In [65]:
trainer.push_to_hub("roberta-sentiment")

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/mspoulaei/best-model-v1/commit/229bb566f65353147894942346350582506bcf9e', commit_message='roberta-sentiment', commit_description='', oid='229bb566f65353147894942346350582506bcf9e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mspoulaei/best-model-v1', endpoint='https://huggingface.co', repo_type='model', repo_id='mspoulaei/best-model-v1'), pr_revision=None, pr_num=None)